<a href="https://colab.research.google.com/github/EmanElrefai/Islamic-chatbot/blob/master/Islamic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Utilization*

In [1]:
import nltk 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
import json
import pickle


import numpy as np
import random
from keras.models import Sequential
from keras.layers import Activation,Dropout,Dense
from keras.optimizers import SGD
from keras.models import load_model

#chatbot gui
from termcolor import colored




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [14]:
ls

intents.json  nltk_data/


# Process data

In [0]:
words=[]
documents=[]
classes=[]
ignore_words=['?','!','.','“', '”','(', ')', ',','’']

data_file = open('intents.json',)
intents=json.load(data_file)


In [16]:
nltk.download('wordnet')

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w=nltk.word_tokenize(pattern)
    words.extend(w)

    documents.append((w,intent['tag']))
    # add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

#lemizer and lower each word and remove duplicates
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))

#sort classes
classes=sorted(list(set(classes)))

#decuments= pattern , intents
print(len(documents),"documents")
#classes= intents
print(len(classes),"classes",classes)
#words = each words , vocabulary
print(len(words),"unique lemmatized words",words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
99 documents
67 classes ['AL-Wahid', 'Al-Arsh', 'Allah with us', "Allah's willing", 'Allahumma Salli', 'Ar-Raziq', 'Ash-hadu', 'Barak Allahu fik', 'Belief in Allah', 'Day of Judgment', 'Ibadah', 'Iman', 'Jinns', "Juzu'", 'Khalifas', 'Kufr', 'Messenger of Allah?', 'Muslim woman', 'Prophet’s parents', 'Prophet’s tribe', 'Qadar', 'Qur’an', 'Rubb', 'Taghut', 'Tauhid', 'Tauhid-al-Asma was-Sifat', 'Tauhid-al-Uluhiyah', 'Tauhid-ar-Rububiyah', 'The master of the Ayahs', "allah's place", 'angels', 'aspects of Tauhid', 'best day', 'best month', 'conditions of Ibadah', 'creation', 'elite jurists', 'evidence of allah', 'foundational sources', 'friends of Allah', 'goodbye', 'grave', 'greater polytheism', 'greatest thing', 'greeting', 'hypocrisy', 'inconspicuous polytheism', 'islam religion', 'last Juzu', 'lesser polytheism?', 'major Kufr', 'no God', 'noanswer', 'not decay', 'number of Taghut',

# Training data

In [17]:

#create training data
training=[]

#create output  empty array
output_=[0]*len(classes)


#bag of words algorithm
for doc in documents:
    #intialize bag for each word
    bag=[]

    pattern_word=doc[0]

    #lemmatize and lower each word in pattern_word
    pattern_word=[lemmatizer.lemmatize(word.lower()) for word in pattern_word]

    for w in words:
      bag.append(1) if w in pattern_word else bag.append(0)

    #output 
    output_row=list(output_)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])

#shuffel features and turn into array
random.shuffle(training)
training=np.array(training)

#train_x is input and train_y is output
train_x=list(training[:,0])
train_y=list(training[:,1])

print('training data created')



training data created


In [18]:
#building sequential model with 3 layers 

model=Sequential()
#first layer with 128 neurons
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
#second layer with 64 neurons
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(len(train_y[0]),activation='softmax'))

# stochastic gradient decsent
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
#compiling model
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

#fitting model
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1,validation_split=0.2)
#saving model
model.save('chatbot_model.h5',hist)

print('model created')




Train on 79 samples, validate on 20 samples
Epoch 1/200
79/79 [==============================] - 0s 4ms/step - loss: 4.2212 - accuracy: 0.0000e+00 - val_loss: 4.1846 - val_accuracy: 0.0000e+00
Epoch 2/200
79/79 [==============================] - 0s 385us/step - loss: 4.1474 - accuracy: 0.0506 - val_loss: 4.2112 - val_accuracy: 0.0000e+00
Epoch 3/200
79/79 [==============================] - 0s 496us/step - loss: 4.1253 - accuracy: 0.0633 - val_loss: 4.2395 - val_accuracy: 0.0000e+00
Epoch 4/200
79/79 [==============================] - 0s 473us/step - loss: 4.0506 - accuracy: 0.0886 - val_loss: 4.2634 - val_accuracy: 0.0000e+00
Epoch 5/200
79/79 [==============================] - 0s 399us/step - loss: 3.9866 - accuracy: 0.0759 - val_loss: 4.2973 - val_accuracy: 0.0000e+00
Epoch 6/200
79/79 [==============================] - 0s 440us/step - loss: 3.8968 - accuracy: 0.1013 - val_loss: 4.3541 - val_accuracy: 0.0000e+00
Epoch 7/200
79/79 [==============================] - 0s 366us/step - los

#predicting the response

In [0]:
#loading words and classes files

words=pickle.load(open("words.pkl","rb"))
classes=pickle.load(open("classes.pkl","rb"))

In [0]:
def clean_up_sentence(sentence):
  #toknize sentence
  sentence_words=nltk.word_tokenize(sentence)
  #lemmtaized and lower each word
  sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]

  return sentence_words

def bow(sentence,words,show_details=False):
  #cleaning up sentence
  sentence_words=clean_up_sentence(sentence)

  bag=[0]*len(words)

  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        # bag equal one if current word is in the vocabulary position
        bag[i]=1

        if show_details==True:
          print('Found in %s',w)

  return np.array(bag)

In [0]:
def predict_class(sentence,model):
  #filter prediction that below threshold
  p=bow(sentence,words,show_details=False)
  res=model.predict(np.array([p]))[0]

  ERROR_THRESHOLD=0.25

  results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  
  #sort by strong probability
  results.sort(key=lambda x:x[1],reverse=True)
  
  return_list=[]
  for r in results:
      return_list.append({"intent":classes[r[0]],"probability":str(r[1])})
  
  
  return return_list






# Running chatbot

In [0]:
def getResponse(ints,intents_json):
  tag=ints[0]['intent']
  list_intents=intents_json['intents']

  for i in list_intents:

    if (tag==i['tag']):
      result=random.choice(i['responses'])
      break

  return result


def chatbot_response(msg):
  ints=predict_class(msg,model)
  res=getResponse(ints,intents)

  return res



In [0]:
def run():
  print(colored("WELCOME TO ISLAMIC CHATBOT", 'red'))
  print("")
  while(1):
     request=input("")
     response=chatbot_response(request)
     print(colored(response, 'cyan'))
     if request=="goodbye"or request== "Bye" or request=="See you later" or request== "Goodbye" or request=="Nice chatting to you, bye" or request=="Till next time":
           break



           

      
     


In [29]:
run()

WELCOME TO ISLAMIC CHATBOT

hi
Good to see you again
how can you help me?
I can guide you to know more about Islam. If you are looking to learn more about Islam whether or not you are a Muslim, or know somebody who is afraid of it, then this is for you
great!! what is islam?
It is submission and obedience to the Order oF Allah and His Messenger with love, hope and fear.
but how to worship allah?
I worship Allah in a manner in which all my ibadah is dedicated to Him Alone. I do not ascribe anyone with Him in worship
thank you so much for great help
My pleasure
Bye
Bye! Come back again soon.
